In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

## UNETR (Transformer)

In [2]:
from monai.networks.nets import UNETR, DynUNet
import torch

if __name__=="__main__":
    
    # for single channel input 4-channel output with image size of (96,96,96), feature size of 32 and batch norm
    net = UNETR(in_channels=5,
                out_channels=2,
                img_size=256,
                feature_size=16,
                dropout_rate=0.3,
                norm_name='batch', 
                spatial_dims=2)

    device = torch.device("cuda: 0") if torch.cuda.is_available() else "cpu"
    x = torch.randn(2, 5, 256, 256).to(device)
    net = net.to(device)

    outs = net(x)
    print(outs.size())


/data/swk/.conda/envs/torch38/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


torch.Size([2, 2, 256, 256])


## Dynamic UNet

In [5]:
if __name__=="__main__":
    
    deep_supervision = False
    
    # without supervision
    net = DynUNet(spatial_dims=2, 
                  in_channels=5, 
                  out_channels=2,
                  kernel_size=[3, 3, 3, 3, 3],
                  upsample_kernel_size=[3, 3, 3, 3, 3],
                  filters=[32, 64, 128, 256, 512],
                  strides=(2, 2, 2, 2, 2),
                  dropout=0.3,
                  deep_supervision=deep_supervision,
                  res_block=True)

    device = torch.device("cuda: 0") if torch.cuda.is_available() else "cpu"
    x = torch.randn(2, 5, 256, 256).to(device)
    net = net.to(device)

    outs = net(x)
    print(outs.size())



RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 24 but got size 16 for tensor number 1 in the list.

## UNet++
(Nested U-Net)

>> https://github.com/4uiiurz1/pytorch-nested-unet

In [6]:
import torch
from torch import nn

__all__ = ['UNet', 'NestedUNet']


class VGGBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels, do_rate=0.3):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, middle_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(middle_channels)
        self.conv2 = nn.Conv2d(middle_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.drop1 = nn.Dropout2d(p=do_rate)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.drop1(out)

        return out


class NestedUNet(nn.Module):
    def __init__(self, 
                 num_classes, 
                 input_channels=3, 
                 deep_supervision=False, 
                 dropout_rate=0.3,
                 nb_filter=[32, 64, 128, 256, 512],
                 **kwargs):
        super().__init__()

        self.deep_supervision = deep_supervision

        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = VGGBlock(input_channels, nb_filter[0], nb_filter[0], do_rate=dropout_rate)
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1], do_rate=dropout_rate)
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2], do_rate=dropout_rate)
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3], do_rate=dropout_rate)
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4], do_rate=dropout_rate)

        self.conv0_1 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0], do_rate=dropout_rate)
        self.conv1_1 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1], do_rate=dropout_rate)
        self.conv2_1 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2], do_rate=dropout_rate)
        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3], do_rate=dropout_rate)

        self.conv0_2 = VGGBlock(nb_filter[0]*2+nb_filter[1], nb_filter[0], nb_filter[0], do_rate=dropout_rate)
        self.conv1_2 = VGGBlock(nb_filter[1]*2+nb_filter[2], nb_filter[1], nb_filter[1], do_rate=dropout_rate)
        self.conv2_2 = VGGBlock(nb_filter[2]*2+nb_filter[3], nb_filter[2], nb_filter[2], do_rate=dropout_rate)

        self.conv0_3 = VGGBlock(nb_filter[0]*3+nb_filter[1], nb_filter[0], nb_filter[0], do_rate=dropout_rate)
        self.conv1_3 = VGGBlock(nb_filter[1]*3+nb_filter[2], nb_filter[1], nb_filter[1], do_rate=dropout_rate)

        self.conv0_4 = VGGBlock(nb_filter[0]*4+nb_filter[1], nb_filter[0], nb_filter[0], do_rate=dropout_rate)

        if self.deep_supervision:
            self.final1 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final2 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final3 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final4 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
        else:
            self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)


    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))

        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            output4 = self.final4(x0_4)
            return [output1, output2, output3, output4]

        else:
            output = self.final(x0_4)
            return output
        
        
if __name__=="__main__":
    
    net = NestedUNet(num_classes=2,
                     input_channels=5,
                     nb_filter=[32, 64, 128, 256, 512],
                     dropout_rate=0.3,
                     deep_supervision=True)

    device = torch.device("cuda: 0") if torch.cuda.is_available() else "cpu"
    x = torch.randn(2, 5, 256, 256).to(device)
    net = net.to(device)

    outs = net(x)
    for out in outs:
        print(out.size())


torch.Size([2, 2, 256, 256])
torch.Size([2, 2, 256, 256])
torch.Size([2, 2, 256, 256])
torch.Size([2, 2, 256, 256])
